In [2]:
from ollama import chat
from pydantic import BaseModel, Field

In [3]:
example = [
        {
            "seqid": 26428,
            "ts": "2024-01-01T00:44:02.697480Z",
            "user": "ubotu",
            "message": "Please watch your language and keep this channel family friendly.",
            "classification": {
                "label": "LABEL_1",
                "score": 0.8133786916732788
            }
        },
        {
            "seqid": 26461,
            "ts": "2024-01-01T00:44:05.997477Z",
            "user": "morghanphoenix",
            "message": "I think I might go with sea monkey someday though.",
            "classification": {
                "label": "LABEL_1",
                "score": 0.8551158905029297
            }
        },
        {
            "seqid": 26563,
            "ts": "2024-01-01T00:44:16.197467Z",
            "user": "amayera",
            "message": "I will try #ubuntu. should be the same issue there.",
            "classification": {
                "label": "LABEL_1",
                "score": 0.862713098526001
            }
        },
        {
            "seqid": 26652,
            "ts": "2024-01-01T00:44:25.097458Z",
            "user": "morghanphoenix",
            "message": "Hey folks, been meaning to discuss that package management workflow we were talking about earlier",
            "classification": {
                "label": "LABEL_1",
                "score": 0.8590824007987976
            }
        },
        {
            "seqid": 26657,
            "ts": "2024-01-01T00:44:25.597458Z",
            "user": "morghanphoenix",
            "message": "How's Thursday evening looking? I can do around 7pm EST",
            "classification": {
                "label": "LABEL_1",
                "score": 0.8530139327049255
            }
        },
        {
            "seqid": 26659,
            "ts": "2024-01-01T00:44:25.797458Z",
            "user": "Dr_willis",
            "message": "I can do 7pm EST, no problem",
            "classification": {
                "label": "LABEL_1",
                "score": 0.8335193395614624
            }
        },
        {
            "seqid": 26719,
            "ts": "2024-01-01T00:44:31.797452Z",
            "user": "cox377",
            "message": "basically, i want to use the cam as a CCTV and have it on a linux box so it starts taking video/images and uploading them as soon as it detects movement, is there something out there for linux that suitable to these requirements?",
            "classification": {
                "label": "LABEL_1",
                "score": 0.8030205965042114
            }
        },
        {
            "seqid": 26737,
            "ts": "2024-01-01T00:44:33.597450Z",
            "user": "vss",
            "message": "I am having problems with kubunte kde menu and crossover. can someone help?",
            "classification": {
                "label": "LABEL_1",
                "score": 0.8442466855049133
            }
        },
        {
            "seqid": 26794,
            "ts": "2024-01-01T00:44:39.297445Z",
            "user": "cox377",
            "message": "does anyone have any recommendations for the best webcam in the budget range that works on linux?",
            "classification": {
                "label": "LABEL_1",
                "score": 0.8126577138900757
            }
        },
        {
            "seqid": 26855,
            "ts": "2024-01-01T00:44:45.397439Z",
            "user": "_morphius",
            "message": "LjL: I really need something I can grep. I am doing scripting. Any other suggestions?",
            "classification": {
                "label": "LABEL_1",
                "score": 0.8559602499008179
            }
        },
        {
            "seqid": 27077,
            "ts": "2024-01-01T00:45:07.597418Z",
            "user": "jende",
            "message": "Jucato: see ya tomorrow?",
            "classification": {
                "label": "LABEL_1",
                "score": 0.8392596244812012
            }
        }
    ]




In [4]:
import uuid

from pydantic import UUID4

class UserMessage(BaseModel):
    '''describing the structure of user messages'''
    user: str
    message: str


class Response(BaseModel):
    '''output describing whether the new message is part of the previous conversation or not'''
    previous_conversation: list[UserMessage]
    new_message: UserMessage
    matches: bool
    reason: str


class Conversation(BaseModel):
    '''store list of user messages as coherent conversations'''
    lines: list[UserMessage]
    id: UUID4 = Field(default_factory=uuid.uuid4)
    

In [5]:
channel_conversation: list[UserMessage] = [{'user': mes['user'], "message": mes['message']}  for mes in example]
channel_conversation

[{'user': 'ubotu',
  'message': 'Please watch your language and keep this channel family friendly.'},
 {'user': 'morghanphoenix',
  'message': 'I think I might go with sea monkey someday though.'},
 {'user': 'amayera',
  'message': 'I will try #ubuntu. should be the same issue there.'},
 {'user': 'morghanphoenix',
  'message': 'Hey folks, been meaning to discuss that package management workflow we were talking about earlier'},
 {'user': 'morghanphoenix',
  'message': "How's Thursday evening looking? I can do around 7pm EST"},
 {'user': 'Dr_willis', 'message': 'I can do 7pm EST, no problem'},
 {'user': 'cox377',
  'message': 'basically, i want to use the cam as a CCTV and have it on a linux box so it starts taking video/images and uploading them as soon as it detects movement, is there something out there for linux that suitable to these requirements?'},
 {'user': 'vss',
  'message': 'I am having problems with kubunte kde menu and crossover. can someone help?'},
 {'user': 'cox377',
  'm

In [6]:
def classify_message(previous_messages: list[UserMessage], msg: UserMessage, model: str) -> Response:
    prompt = f"""
        You are provided with a conversation with user and the message they wrote in a IRC or slack channel you need to tell if the given new message is part of the conversation or not
        
        Conversation: 
        {previous_messages}

        New Message:
        {msg}
    """
    response = chat(
        messages = [
            {'role': 'user', 'content': prompt},
        ],
        model=model,
        format=Response.model_json_schema(),
        options={
            'temperature': 0
        }
    )
    return Response.model_validate_json(response.message.content)


In [7]:
state = {}

for message in channel_conversation:
    matched_to_conv = False
    for conv_id in state.keys():
        conv = state[conv_id]
        resp = classify_message(previous_messages=conv.lines, msg=message, model="openthinker:7b")
        if resp.matches:
            print(resp)
            conv.lines.append(message)
            state[conv.id] == conv
            matched_to_conv = True
            break
    if not matched_to_conv:
        conv = Conversation(lines=[message])
        state[conv.id] = conv


previous_conversation=[UserMessage(user='ubotu', message='Please watch your language and keep this channel family friendly.')] new_message=UserMessage(user='amayera', message='I will try #ubuntu. should be the same issue there.') matches=True reason="The new message references '#ubuntu' which is a common channel name, and the context of the previous message was about maintaining a family-friendly environment. The user 'amayera' is suggesting moving to #ubuntu where they believe the same issue exists."
previous_conversation=[UserMessage(user='morghanphoenix', message='I think I might go with sea monkey someday though.')] new_message=UserMessage(user='morghanphoenix', message='Hey folks, been meaning to discuss that package management workflow we were talking about earlier') matches=True reason="The new message references a previous discussion ('package management workflow') mentioned in the initial conversation. The user 'morghanphoenix' is the same in both messages, and the context imp

In [8]:
state

{UUID('3348dcc0-abca-4687-8085-22773450369e'): Conversation(lines=[UserMessage(user='ubotu', message='Please watch your language and keep this channel family friendly.'), {'user': 'amayera', 'message': 'I will try #ubuntu. should be the same issue there.'}], id=UUID('3348dcc0-abca-4687-8085-22773450369e')),
 UUID('616921a9-cfd7-4062-82eb-484cdac06c00'): Conversation(lines=[UserMessage(user='morghanphoenix', message='I think I might go with sea monkey someday though.'), {'user': 'morghanphoenix', 'message': 'Hey folks, been meaning to discuss that package management workflow we were talking about earlier'}], id=UUID('616921a9-cfd7-4062-82eb-484cdac06c00')),
 UUID('736f5a7d-9f6f-4488-b543-28ea7bfb6c3c'): Conversation(lines=[UserMessage(user='morghanphoenix', message="How's Thursday evening looking? I can do around 7pm EST"), {'user': 'Dr_willis', 'message': 'I can do 7pm EST, no problem'}], id=UUID('736f5a7d-9f6f-4488-b543-28ea7bfb6c3c')),
 UUID('8058d352-b148-466f-a817-26596b058205'): C

conv1
```json
  {'user': 'ubotu',
   'message': 'Please watch your language and keep this channel family friendly.'}
 ```

conv 2
```json 
  {'user': 'morghanphoenix',
   'message': 'I think I might go with sea monkey someday though.'},
```

conv 3
```json
{'user': 'amayera',
  'message': 'I will try #ubuntu. should be the same issue there.'},
```

conv 4
```json
 {'user': 'morghanphoenix',
  'message': 'Hey folks, been meaning to discuss that package management workflow we were talking about earlier'},
 {'user': 'morghanphoenix',
  'message': "How's Thursday evening looking? I can do around 7pm EST"},
 {'user': 'Dr_willis', 'message': 'I can do 7pm EST, no problem'},
```

conv 5
```json
 {'user': 'cox377',
  'message': 'basically, i want to use the cam as a CCTV and have it on a linux box so it starts taking video/images and uploading them as soon as it detects movement, is there something out there for linux that suitable to these requirements?'},
 {'user': 'cox377',
  'message': 'does anyone have any recommendations for the best webcam in the budget range that works on linux?'},
```

conv 6
```json
 {'user': 'vss',
  'message': 'I am having problems with kubunte kde menu and crossover. can someone help?'},
```


conv 7
```json
 {'user': '_morphius',
  'message': 'LjL: I really need something I can grep. I am doing scripting. Any other suggestions?'},
```

conv 8
```json
 {'user': 'jende', 'message': 'Jucato: see ya tomorrow?'}
```